In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pd_support import read_df_csv, add_datetime_col
from scipy.interpolate import interp1d

In [2]:
cart_df = pd.read_csv("..//data_csv//sync_test.csv")
cart_df

,sys_time,e_fr,e_fl,e_rr,e_rl,rtc,mils,sync
0,2022-06-22 12:22:54.161451,-2791,3329,-7936,7161,2022-06-22 05.52.51.000000 PM,460040,0
1,2022-06-22 12:22:54.161451,-2791,3329,-7936,7161,2022-06-22 05.52.51.000000 PM,460050,0
2,2022-06-22 12:22:54.161947,-2791,3329,-7936,7161,2022-06-22 05.52.51.000000 PM,460060,0
3,2022-06-22 12:22:54.162451,-2791,3329,-7936,7161,2022-06-22 05.52.51.000000 PM,460070,0
4,2022-06-22 12:22:54.162950,-2791,3329,-7936,7161,2022-06-22 05.52.51.000000 PM,460080,0
...,...,...,...,...,...,...,...,...
3165,2022-06-22 12:23:24.715272,-3890,4209,-7877,6763,2022-06-22 05.53.23.000000 PM,491712,0
3166,2022-06-22 12:23:24.715774,-3890,4209,-7877,6763,2022-06-22 05.53.23.000000 PM,491722,0
3167,2022-06-22 12:23:24.716272,-3890,4209,-7877,6763,2022-06-22 05.53.23.000000 PM,491732,0
3168,2022-06-22 12:23:24.716771,-3890,4209,-7877,6763,2022-06-22 05.53.23.000000 PM,491742,0


In [3]:
for inx, i in enumerate(cart_df["sync"]):
    if i == 1:
        print(inx)
        break
cart_df.iloc[inx]

431


sys_time       2022-06-22 12:22:57.381846
e_fr                                -2791
e_fl                                 3329
e_rr                                -7936
e_rl                                 7161
rtc         2022-06-22 05.52.56.000000 PM
mils                               464353
sync                                    1
Name: 431, dtype: object

In [4]:
mc, mc_dt = read_df_csv("..//data_csv//cart_random_sync_mc.csv")
mc_dt

datetime.datetime(2022, 6, 22, 12, 22, 57, 287000)

In [5]:
mc = add_datetime_col(mc, mc_dt, "seconds")
mc

,frame,seconds,org_x,org_y,org_z,xdir_x,xdir_y,xdir_z,zdir_x,zdir_y,zdir_z,time
0,0,0.00,0.186409,0.066477,0.243783,0.227569,0.065008,0.251833,0.169555,0.066373,0.345257,2022-06-22 12:22:57.287
1,1,0.01,0.186456,0.066552,0.243839,0.227555,0.065018,0.251839,0.169555,0.066388,0.345246,2022-06-22 12:22:57.297
2,2,0.02,0.186459,0.066578,0.243845,0.227562,0.065023,0.251848,0.169556,0.066387,0.345249,2022-06-22 12:22:57.307
3,3,0.03,0.186411,0.066453,0.243761,0.227565,0.065003,0.251846,0.169567,0.066400,0.345258,2022-06-22 12:22:57.317
4,4,0.04,0.186466,0.066558,0.243862,0.227568,0.065005,0.251848,0.169557,0.066386,0.345249,2022-06-22 12:22:57.327
...,...,...,...,...,...,...,...,...,...,...,...,...
2483,2483,24.83,0.143248,0.067062,0.242522,0.185217,0.065387,0.241843,0.147989,0.066924,0.345371,2022-06-22 12:23:22.117
2484,2484,24.84,0.143246,0.067066,0.242524,0.185204,0.065339,0.241992,0.147995,0.066919,0.345369,2022-06-22 12:23:22.127
2485,2485,24.85,0.143245,0.067056,0.242524,0.185221,0.065387,0.241835,0.147997,0.066912,0.345366,2022-06-22 12:23:22.137
2486,2486,24.86,0.143252,0.067062,0.242526,0.185217,0.065378,0.241833,0.147995,0.066914,0.345370,2022-06-22 12:23:22.147


In [6]:
def add_datetime_diff(df, _time, _sync, _diff_name, truncate = False):

    """
    df:         dataframe
    _time:      the time you want to start your column with
    _sync:      Name of the column with external sync which turns 1 when your recording 
                your data and 0 when your not
    _diff_name: external time/ other time which you want to find the difference 
                and add them to the _time column to sync clock
    truncate:   truncate will also cut the values after your _sync goes to 0
    """
    _inx = 0
    for inx, i in enumerate(df[_sync]):
        if i == 1:
            _inx = inx
            break

    df = df.iloc[_inx:]     # dropping unnecessary rows

    _diff = list(df[_diff_name].diff())
    _sum = 0
    _t = []
    for i in _diff:
        if np.isnan(i):
            i = 0
        _sum = _sum + i
        _t.append(_time + timedelta(0,float(_sum/1000)))

    df["time"] = _t

    if truncate:
        m1 = 0
        _count = 0
        for count, j in enumerate(df[_sync]):
            print(count)
            print(j)
            m1+=1
            if j == 0:
                _count = count
                print(_count)
            break
    
    
        df1 = df.iloc[:_count]     # dropping unnecessary rows
        print(df1)
    
    return df

In [7]:
cart_df = add_datetime_diff(cart_df, mc_dt,"sync", "mils", True)
# cart_df = add_datetime_diff(cart_df, mc_dt,"sync", "mils", False)
cart_df

0
1
Empty DataFrame
Columns: [sys_time, e_fr, e_fl, e_rr, e_rl, rtc, mils, sync, time]
Index: []


C:\Users\CMC\anaconda3\envs\py37t2\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,sys_time,e_fr,e_fl,e_rr,e_rl,rtc,mils,sync,time
431,2022-06-22 12:22:57.381846,-2791,3329,-7936,7161,2022-06-22 05.52.56.000000 PM,464353,1,2022-06-22 12:22:57.287
432,2022-06-22 12:22:57.382862,-2791,3329,-7936,7161,2022-06-22 05.52.56.000000 PM,464363,1,2022-06-22 12:22:57.297
433,2022-06-22 12:22:57.383344,-2791,3329,-7936,7161,2022-06-22 05.52.56.000000 PM,464373,1,2022-06-22 12:22:57.307
434,2022-06-22 12:22:57.383846,-2791,3329,-7936,7161,2022-06-22 05.52.56.000000 PM,464383,1,2022-06-22 12:22:57.317
435,2022-06-22 12:22:57.384344,-2791,3329,-7936,7161,2022-06-22 05.52.56.000000 PM,464393,1,2022-06-22 12:22:57.327
...,...,...,...,...,...,...,...,...,...
3165,2022-06-22 12:23:24.715272,-3890,4209,-7877,6763,2022-06-22 05.53.23.000000 PM,491712,0,2022-06-22 12:23:24.646
3166,2022-06-22 12:23:24.715774,-3890,4209,-7877,6763,2022-06-22 05.53.23.000000 PM,491722,0,2022-06-22 12:23:24.656
3167,2022-06-22 12:23:24.716272,-3890,4209,-7877,6763,2022-06-22 05.53.23.000000 PM,491732,0,2022-06-22 12:23:24.666
3168,2022-06-22 12:23:24.716771,-3890,4209,-7877,6763,2022-06-22 05.53.23.000000 PM,491742,0,2022-06-22 12:23:24.676
